# Ferramentas de Análise de Paralelismo - Intel Advisor
### Importante : 
Para execução deste material, é preciso a instalação do [_HPC Intel Toolkit_](https://www.intel.com/content/www/us/en/developer/tools/oneapi/hpc-toolkit-download.html) ou pelo uso da __Intel Devcloud__. Neste jupyter, faremos a demonstração e explicação do roofline por meio de dois exemplos retirados [deste repositorio](https://github.com/oneapi-src/oneAPI-samples):
- Mandelbrot
- DiscreteCosineTransform

## Oque é Roofline ? 
Roofline é uma representação visual do desempenho do algoritmo em relação ao hardware no qual é executado, oque permite uma análise visual de largura de banda de memória e picos computacionais. O intel Advisor é uma das ferramentas que mede e plota este tipo de gráfico automatizadamente.

## Requisitos para GPU's 
Para realizar uma implementação em GPU, é necessario gráficos integrados - de Geração 9 ou 11 - e a transmissao destes dados deve ser efetuada por meio das linguagens de programação _OpenMP, SYCL, DPC++_ ou _OpenCL_. 

## Como é o gráfico funciona, na prática? 

No exemplo abaixo, temos a apresentação de um gráfico de Roofline. Ele é composto principalmente por duas linhas que são os limitantes do hardware em questão, sendo um limitante de Largura de Banda e outro Limitante de Intensidade Operacional. Vamos abordar cada um: 


In [ ]:
import os
from IPython.display import IFrame
os.system(' Terminal de execucao :/bin/echo $(whoami)\n Analise de Roofline -- roofline.html')
IFrame(src='assets/roofline.html', width=1024, height=769)

- `Largura de Memória(Bandwidth):`
Temos a representação de _bandwidth_ como a taxa máxima de transferência de dados entre Memória e CPU/GPU esta unidade de transferência de dados é importante para algoritmos em paralelos, justamente porque um dos gargalos que podemos ter em nossos algoritmos é a memória. Esta linha é representada pela linha diagonal principal.


- `Intensidade Operacional(I.O):`
Apresenta a quantidade de operações artiméticas por byte transferido. Ou seja, apresenta o quanto o processador está "ocupado" realizando operações e sua intensidade de acordo com o total de dados recebidos. Vale ressaltar que sua métrica é apresentada pela razão de operações máximas de ponto flutuante(FLOPS) por byte recebido. Esta medida é representa pela linha horizontal.


Note que temos mais de uma linha que representa a `largura de banda` e `Intensidade Operacional`. Isso porque , em um mesmo hardware, temos diferentes modos de acesso à memória(Acesso a Memória RAM, cache L1, L2, L3), bem como operações que possuem tamanhos distintos (sizeof float = 32 bits, sizeof double = 64, etc..). Em resumo, temos que levar em conta o tipo de dado que estamos utilizando na nossa operação para determinarmos qual topo teórico será levado em consideração. 

<img src='assets/r2.png'>

##  Gerando Rooflines com exemplos da OneAPI
Neste notebook, escolhemos os seguintes exemplos do repositório da oneAPI: 
- Exemplo 01: Discrete Cossine Transformation (DCT)
- Exemplo 02: Sparse Matrix Multiplicação 

Abordaremos as nuances de cada algoritmo abaixo. Siga as instruções em cada célula.

In [ ]:
%%writefile DiscreteCosineTransform/src/CMakeLists.txt
# precisamos modificar o CMAkeLists para adicionar um novo target: "run-profile"

if(PERF_NUM)
set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} -g -fsycl -std=c++17 -D PERF_NUM")
message (STATUS "target will be built for performance tabulation")
else()
set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} -fsycl -std=c++17")
endif()
set(CMAKE_EXE_LINKER_FLAGS "${CMAKE_EXE_LINKER_FLAGS}")
add_executable (dct DCT.cpp)
target_link_libraries(dct OpenCL sycl)
file(COPY ../res/willyriver.bmp DESTINATION .)
add_custom_target (run ./dct willyriver.bmp willyriver_processed.bmp)

add_custom_target(run-profile
  COMMAND advisor --collect=roofline --project-dir=./adv --  ./dct willyriver.bmp willyriver_processed.bmp
  COMMAND advisor --report=roofline --project-dir=./adv --report-output=./../../../dct_roofline.html
  DEPENDS dct
  COMMENT "Running Intel Advisor on dct")


In [ ]:
%%writefile DiscreteCosineTransform/build_run.sh

source /opt/intel/inteloneapi/setvars.sh > /dev/null 2>&1s
# Advisor env-variables
source /opt/intel/inteloneapi/advisor/2023.0.0/advisor-vars.sh
source /opt/intel/inteloneapi/advisor/2023.0.0/advixe-vars.sh

#Build by CMAKE
mkdir build
cd build
cmake ..
make

make run-profile


In [ ]:
# Execute esta célula para submeter o Exemplo 01.
!cd DiscreteCosineTransform && chmod +x build_run.sh && ./build_run.sh;

In [ ]:
%%writefile mandelbrot/src/CMakeLists.txt
# precisamos modificar o CMAkeLists para adicionar um novo target: "run-profile"

set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} -g -std=c++17 -fsycl")
set(CMAKE_EXE_LINKER_FLAGS "${CMAKE_EXE_LINKER_FLAGS}")

add_executable(mandelbrot main.cpp)
target_link_libraries(mandelbrot OpenCL sycl)
add_custom_target(run ./mandelbrot)

add_executable(mandelbrot_usm main.cpp)
target_compile_definitions(mandelbrot_usm PRIVATE MANDELBROT_USM)
target_link_libraries(mandelbrot_usm OpenCL sycl)
add_custom_target(run_usm ./mandelbrot_usm)

add_custom_target(run-profile
  COMMAND advisor --collect=roofline --flop --project-dir=./adv -- ./mandelbrot_usm
  # direciona o arquivo para a pasta SYCL                
  COMMAND advisor --report=roofline --project-dir=./adv --report-output=./../../../mandel_roofline.html
  DEPENDS mandelbrot_usm
  COMMENT "Running Intel Advisor on MandelBrot_USM"
)


In [ ]:
%%writefile mandelbrot/build_run.sh

source /opt/intel/inteloneapi/setvars.sh > /dev/null 2>&1s
# Advisor ebuild_runriables
source /opt/intel/inteloneapi/advisor/2023.0.0/advisor-vars.sh
source /opt/intel/inteloneapi/advisor/2023.0.0/advixe-vars.sh

#Build by CMAKE
mkdir build
cd build
cmake ..
make

make run-profile

In [ ]:
# Execute esta célula para submeter o build do Exemplo 02.
!cd mandelbrot && chmod +x build_run.sh && ./build_run.sh

Com a execução das células acima, você notará a criação de dois arquivos na pasta principal :
- dct_roofline.html 
- mandel_roofline.html 

Estes arquivos podem ser visualizados aqui mesmo, nas células logo abaixo:

In [ ]:
#Impressao Roofline de dct
from IPython.display import IFrame
IFrame('dct_roofline.html', width='100%', height=600)


In [ ]:
#Impressao Roofline de MandelBrot
from IPython.display import IFrame
IFrame(src='mandel_roofline.html', width='100%', height=769)

## Faça voce mesmo !
Agora é a sua vez, faça o upload de algum dos exemplos do [Repositório INTEL](https://github.com/oneapi-src/oneAPI-samples/tree/master/DirectProgramming/C%2B%2BSYCL) e faça uma breve análise do resultado obtido de acordo com o caso selecionado !
